<a href="https://colab.research.google.com/github/MartinKugler97/PLN/blob/main/3_extraccion_grupo_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# APARTADO 3 - EXTRACCIÓN DE INFORMACIÓN
### Autores: Diego Recover y Martín Kugler (Grupo 09)

In [1]:
import requests # Usado para realizar peticiones a páginas web para descargarlas.
from bs4 import BeautifulSoup # Usado para navegar en archivos html y extraer datos.

## TAREA 1: Descarga de las páginas web y extracción de los titulares correspondientes.

#### I) Descargar páginas web:

In [2]:
def descargar_html(url: str, timeout=10):
    '''
    Dada una URL, descarga la página web en formato HTML.
    El tiempo de espera máximo de la petición se introduce en 'timeout'.
    '''

    # Definimos un User-Agent para simular un navegador y prevenir bloqueos:
    user_agent = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/100.0.0.0 Safari/537.36"
    }

    try:
        # Realizamos la petición GET a la página web:
        respuesta = requests.get(url, headers=user_agent, timeout=timeout)

        # Comprobamos el código de estado (si es distinto de 200, se considera que
        # se rechazó la petición y se devuelve None junto a una advertencia):
        if respuesta.status_code != 200:
            print(f"Advertencia: la petición a {url} devolvió el código de estado: {respuesta.status_code}")
            return None

        # Devolvemos el contenido HTML de la página web:
        return respuesta.text

    except requests.RequestException as re:
        # Esto captura errores de red, timeout, DNS y demás.
        print(f"Error al descargar la URL {url}: {re}")
        return None

In [3]:
# Escogeremos las páginas web de los periódicos de El País y de La Vanguardia:
el_pais_url = 'https://elpais.com'
la_vanguardia_url = 'https://www.lavanguardia.com'

# Obtenemos mediante la función definida sus respectivas páginas web en formato HTML:
el_pais_html = descargar_html(el_pais_url)
la_vanguardia_html = descargar_html(la_vanguardia_url)

#### II) Extraer los titulares:

In [4]:
def extraer_titulares(html, max_titulares=100):
    '''
    Dada una página web en formato HTML, extrae los titulares de dicha página web
    y los devuelve en una lista.
    El número máximo de titulares extraídos se define en 'max_titulares'.
    '''

    # Si no se ha podido descargar el HTML (None), devolvemos lista vacía:
    if html is None:
        return []

    # Creamos el objeto BeautifulSoup para analizar el documento HTML:
    soup = BeautifulSoup(html, "html.parser")

    # Inicializamos la lista donde guardaremos los titulares que vayamos encontrando:
    titulares = []

    # Inicializamos un conjunto para evitar duplicados:
    vistos = set()

    # Buscamos las etiquetas de HTML típicas de titulares (h1, h2 y h3):
    etiquetas = soup.find_all(["h1", "h2", "h3"])

    # Recorremos cada etiqueta encontrada:
    for tag in etiquetas:

        # Extraemos el texto limpio (sin saltos ni espacios):
        texto = tag.get_text(strip=True)

        # Descartamos textos vacíos o muy cortos (suelen ser ruido):
        if len(texto) < 10:
            continue

        # Evitamos duplicados comprobando si ya lo tenemos en el conjunto "vistos":
        if texto not in vistos:
            vistos.add(texto)
            titulares.append(texto)

        # Si ya alcanzamos el máximo permitido, paramos:
        if len(titulares) >= max_titulares:
            break

    # Devolvemos los titulares:
    return titulares

In [5]:
# Obtenemos los titulares de cada página web:
el_pais_titulares = extraer_titulares(el_pais_html)
la_vanguardia_titulares = extraer_titulares(la_vanguardia_html)

el_pais_titulares, la_vanguardia_titulares

(['El Senado acuerda por unanimidad enviar sin cambios la ley de los papeles de Epstein a Trump para que la firme',
  'El presidente\xa0ha autorizado un plan de acciones encubiertas en Venezuela, según el\xa0‘New York Times’',
  'México eleva el tono ante la insistencia del republicano de una intervención militar',
  'Video | La cifra secreta de la cocaína en Colombia',
  'Antonio Tijerino: “Un joven con una laptop puede alcanzar a más personas en un segundo que Gandhi o Martin Luther King en su vida”',
  'El horror al interior de los albergues Niños de México: más de medio siglo de encubrimiento de abuso sexual y violencia contra menores',
  'Actualidad',
  'Larry Summers, ex secretario del Tesoro de Estados Unidos, dimite como consejero de OpenAI por su conexión con Jeffrey Epstein',
  'Una comisión del Congreso argentino atribuye a Milei “una colaboración imprescindible” en la presunta estafa con $Libra',
  'Detenido uno de los presuntos autores intelectuales del asesinato del alcal

## Tarea 2: Obtención del número de titulares que se refieren a la misma noticia.

#### I) Convertir titulares en vectores:

Existen dos opciones dadas en clase: TF-IDF y Word2Vec (la variante Skip-gram). Esta última tiene el inconveniente de ser más compleja al ser necesario entrenar el modelo, lo cual necesitaría muchos titulares para ser precisa (de los cuales tenemos una cantidad bastante pequeña). Por tanto, debido a esto y a la simplicidad de TF-IDF, escogeremos este método para vectorizar los titulares.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def vectorizar_tfidf(titulares_1: list[str], titulares_2: list[str]) -> tuple:
    '''
    Función que vectoriza empleando la técnica TF-IDF, devolviendo dos matrices
    resultantes para cada periódico.

    A modo de poderse comparar entre sí, se empleará el mismo vectorizador para
    ambos periódicos, de modo que compartirán vocabulario.
    '''

    # Agrupamos ambos coonjuntos de titulares para entrenar el TF-IDF con un único
    # vocabulario conjunto:
    titulares_totales = titulares_1 + titulares_2

    # Creamos el vectorizador TF-IDF y lo ajustamos a los titulares:
    vectorizador = TfidfVectorizer()
    vectorizador.fit(titulares_totales)

    # Transformamos cada grupo de titulares por separado (compartiendo el mismo vocabulario):
    vec_1 = vectorizador.transform(titulares_1)
    vec_2 = vectorizador.transform(titulares_2)

    # Devolvemos las matrices TF-IDF de cada periódico:
    return vec_1, vec_2

In [7]:
# Prueba del código:
el_pais_vec, la_vanguardia_vec = vectorizar_tfidf(el_pais_titulares, la_vanguardia_titulares)
el_pais_vec, la_vanguardia_vec

(<Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 1098 stored elements and shape (100, 1167)>,
 <Compressed Sparse Row sparse matrix of dtype 'float64'
 	with 958 stored elements and shape (100, 1167)>)

#### II) Calcular similaridades:

Para calcular la similaridad entre titulares, haremos uso de la **distancia coseno** vista en clase, la cual se basa en el ángulo entre dos vectores que representan palabras.

Esto es posible gracias a la operación realizada en la función anterior, en la que al unir ambos titulares al crear el vectorizador generamos un vocabulario conjunto para ambos, de modo que ambas matrices TF-IDF comparten el **mismo espacio vectorial** y, por tanto, son comparables.

In [8]:
def calcular_similaridades(vec_1: list, vec_2: list) -> np.ndarray:
    '''
    Función que calcula la matriz de similaridad por coseno entre todos los
    titulares de ambos periódicos vectorizados en vec_1 y vec_2.
    '''

    # Calculamos matriz de similaridad:
    matriz_sim = cosine_similarity(vec_1, vec_2)

    return matriz_sim

In [9]:
# Prueba del código:
matriz_sim = calcular_similaridades(el_pais_vec, la_vanguardia_vec)
matriz_sim


array([[0.06554243, 0.05579032, 0.07218482, ..., 0.        , 0.07005573,
        0.03277121],
       [0.        , 0.01906707, 0.11945315, ..., 0.        , 0.        ,
        0.06641734],
       [0.03849393, 0.02022183, 0.02965442, ..., 0.        , 0.04114465,
        0.03849393],
       ...,
       [0.        , 0.03067925, 0.03443766, ..., 0.        , 0.        ,
        0.02810373],
       [0.03817039, 0.05005579, 0.02940518, ..., 0.        , 0.04079884,
        0.03817039],
       [0.06503266, 0.04568129, 0.0523695 , ..., 0.        , 0.06951086,
        0.        ]])

#### III) Obtener número de titulares similares (que se refieren a la misma noticia):

El paso final que quedaría sería el de contar el número de similaridades en la matriz de similaridades hallada en la función anterior en función de un determinado **umbral**. Pondremos por defecto un umbral de 0.30, sabiendo que estamos usando TF-IDF como vectorizador, que es más limitado en cuanto a entendimiento de semántica.

In [12]:
def num_titulares_sim(matriz_sim: np.ndarray, umbral=0.30) -> int:
    '''
    Función que calcula cuántas parejas en la matriz_sim superan el umbral dado.
    '''
    return np.sum(matriz_sim > umbral)

In [13]:
# Prueba del código:
num_titulares_sim(matriz_sim) # MAL -> POR VER.

np.int64(5)

#### EN PROCESO...

In [ ]:
# Encapsulamos todas las funcionalidades requeridades en una única función final:

def comparar_titulares(url_1: str, url_2: str, umbral=0.80)
    '''
    Función que compara la similaridad de titulares entre dos periódicos según un
    umbral de similaridad dado (por defecto 0.80).
    '''

    # Descargamos los respectivos periódicos y extraemos 100 titulares:
    html_1 = descargar_html(url_1)
    html_2 = descargar_html(url_2)

    titulares_1 = extraer_titulares(html_1)
    titilares_2 = extraer_titulares(html_2)

    # Creamos el vectorizador TF-IDF:
    tfidf_vectorizador = TfidfVectorizer()

    # Transformamos